In [464]:
import math
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [465]:
import graphviz

In [466]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes, edges

In [467]:
def draw_dot(root):
    g = graphviz.Digraph('gradient',format="svg",graph_attr={'rankdir':'LR'})
    nodes, edges = trace(root)
    for n in nodes:
        uid =str(id(n))
        g.node(name=uid,label="%s | data %.4f | grad %.4f" % (n.label,n.data,n.grad),shape='record')
        if n._op:
            g.node(name=uid+n._op,label=n._op)
            g.edge(uid+n._op,uid)
    
    for n1 , n2 in edges:
        g.edge(str(id(n1)),str(id(n2)) + n2._op)
    
    return g

In [468]:
x1 = Value(2.0,label='x1')
x2 = Value(0.0,label='x2')
w1 = Value(-3.0,label='w1')
w2 = Value(1.0,label='w2')
b = Value(6.88137358,label='b')
x1w1 = x1*w1 ; x1w1.label = 'x1w1' 
x2w2 = x2*w2 ; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1+x2w2 ; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b ; n.label = 'n'
o = n.tanh() ; o.label = 'o'
o.backward()


In [469]:
print(o.data)
print(w1.grad)
print(w2.grad)
print(x1.grad)  
print(x2.grad)

0.707106777676776
1.000000009927133
0.0
-1.5000000148906993
0.5000000049635664


In [470]:
## doing the same thing but using pytorch

import torch

x1 = torch.tensor([2.0],requires_grad=True)
x2 = torch.tensor([0.0],requires_grad=True)
w1 = torch.tensor([-3.0],requires_grad=True)
w2 = torch.tensor([1.0],requires_grad=True)

b = torch.tensor([6.88137358],requires_grad=True)

n = x1*w1 + x2*w2 + b 
o = torch.tanh(n)
o.backward()

In [471]:
print(o.data.item())
print(w1.grad.item())
print(w2.grad.item())
print(x1.grad.item())  
print(x2.grad.item())



0.7071067094802856
1.000000238418579
0.0
-1.5000003576278687
0.5000001192092896


In [472]:
import random

class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]


In [473]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)


Value(data=-0.8214020351811855)

In [474]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets


In [475]:
for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)


0 6.964222320634499
1 3.3754510087101375
2 1.9648403153893943
3 0.37820376760646346
4 0.19778085798121886
5 0.13507188156926472
6 0.1011339213252656
7 0.080071749559229
8 0.06584013614016453
9 0.05563943844000325
10 0.04800283708945691
11 0.042091351254011936
12 0.037392249540532496
13 0.033575471188505415
14 0.030419447999479862
15 0.02777020895159591
16 0.02551760623531748
17 0.023580870324033422
18 0.021899498701286733
19 0.020427321250161945
